In [2]:
import os
import sys
cwd = os.getcwd()
src_path = os.path.join(cwd, "src")
sys.path.append(src_path)

In [3]:
from feature_extraction import extract_features_from_parse, dict2parse

crel2probs = {
    "Causer:7->Result:50": [0.5, 0.6],
    "Causer:3->Result:4": [0.6],
    "Causer:4->Result:5": [0.6],
    "Causer:5->Result:4": [0.65],
    
    "Causer:8->Result:1": [0.35],
}

parse = dict2parse(crel2probs)
extract_features_from_parse(parse, crel2probs)

defaultdict(float,
            {'Above-%-0.2': 1.0,
             'Above-%-0.3': 1.0,
             'Above-%-0.5': 0.8,
             'Above-%-0.7': 0.0,
             'Above-%-0.8': 0.0,
             'Above-%-0.9': 0.0,
             'Above-%-0.95': 0.0,
             'Above-0.2': 5,
             'Above-0.3': 5,
             'Above-0.5': 4,
             'Above-0.7': 0,
             'Above-0.8': 0,
             'Above-0.9': 0,
             'Above-0.95': 0,
             'Above-All-Above-0.2': 1,
             'Above-All-Above-0.3': 1,
             'CChain-:3,4,5': 1,
             'CChainStats-MaxChain_Len=3': 1,
             'CChainStats-crels_per_distinct_chain': 5.0,
             'CChainStats-num_distinct_chains=1': 1,
             'CREL_3:4': 1,
             'CREL_4:5': 1,
             'CREL_5:4': 1,
             'CREL_7:50': 1,
             'CREL_8:1': 1,
             'CREL_Causer:3->Result:4-MAX(prob)': 0.6,
             'CREL_Causer:3->Result:4-MIN(prob)': 0.6,
             'CREL_Causer:

In [27]:
from collections import defaultdict
from pprint import pprint

def build_cb_causal_model():
    cm = dict()
    for i in range(1,5):
        cm[str(i)] = str(i+1)
    cm["5"] = "5b"
    cm["5b"] = "14"
    cm["14"] = "6"
    cm["6"] = "7"
    cm["7"] = "50"
    for i in range(11,14):
        cm[str(i)] = str(i+1)
    return cm

def build_sc_causal_model():
    cm = dict()
    for i in range(1,6):
        cm[str(i)] = str(i+1)
    cm["11"] = "12"
    cm["12"] = "6"
    cm["6"] = "50"
    return cm

build_sc_causal_model()

{'1': '2',
 '11': '12',
 '12': '6',
 '2': '3',
 '3': '4',
 '4': '5',
 '5': '6',
 '6': '50'}

In [42]:
from typing import Dict

def to_numeric_tag(tag: str)->int:
    nums = ""
    for c in tag:
        if c.isdigit():
            nums += c
    return int(nums)

TERMINAL = "50"
cb_mdl = build_cb_causal_model()
sc_mdl = build_sc_causal_model()

def distance_between(a:str, b:str, causal_model: Dict[str, str]):
    # ensure a <= b
    a, b = str(a), str(b)
    assert a in causal_model or a == TERMINAL, "a is not in the model"
    assert b in causal_model or b == TERMINAL, "b is not in the model"

    cross_over_node = (a == "6" and b in {"12", "14"})
    reverse_cross_over_node = (b == "6" and a in {"12", "14"})
    if cross_over_node:
        a, b = b, a
    
    if not reverse_cross_over_node and (to_numeric_tag(a) > to_numeric_tag(b)):
        a, b = b, a
        
    print(a,b, cross_over_node)
    return __nodes_between__(a, b, causal_model)

def __nodes_between__(a, b, causal_model):
    nodes_btwn = 0
    current = a
    while current != b and current != TERMINAL:
        current = causal_model[current]
        nodes_btwn += 1
    # couldn't reach the other node
    if b != TERMINAL and current == TERMINAL:
        return -1
    return nodes_btwn


distance_between("6","14", cb_mdl)

6 14 True


-1